## 징역, 집행유예, 벌금 추출

`case_result`에서 징역, 집행유예 여부, 벌금에 대한 정보를 가져온 후 새로운 컬럼에 저장합니다.

In [68]:
import pandas as pd
import re
import xlsxwriter
import os

INPUT_FILE_PATH = "./output/02_data_output.xlsx" # 입력 데이터 파일
OUPUT_PATH = "./output/" # 데이터 출력 경로

# 데이터 가져오기
dataframe = pd.read_excel(INPUT_FILE_PATH)

imprisonments, probations, fines = [], [], [] # 징역(달), 집행유예(년), 벌금

for case_result in dataframe['case_result'][:]:
    imprisonment, probation, fine = 0, 0, 0
    for elem in case_result.split("·"):
        if "징역" in elem:
            raw_imprisonment = re.sub("[(징역)(장기) ]", "", elem)
            year, month, tmp = 0, 0, ""
            for char in raw_imprisonment:
                if char == "년":
                    year = int(tmp)
                    tmp = ""
                elif char == "월":
                    month = int(tmp)
                    tmp = ""
                else:
                    tmp += char
            imprisonment = 12 * year + month
        elif "집행유예" in elem:
            probation = int(re.sub("[(집행유예)(년) ]", "", elem))
        elif "벌금" in elem:
            fine = int(re.sub("[(벌금)(만원), ]", "", elem))

    imprisonments.append(imprisonment) # 징역 추가
    probations.append(probation) # 집행유예 추가
    fines.append(fine) # 벌금 추가

# 징역, 집행유예, 벌금 열 추가
dataframe['imprisonment'] = imprisonments
dataframe['probation'] = probations
dataframe['fine'] = fines

# 엑셀파일로 저장
dataframe.to_excel(os.path.join(OUPUT_PATH, "03_data_output.xlsx"), index=False, engine = "xlsxwriter")